In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

# 初期値

In [2]:
DAYS = 4000 # 生成日数
FROM_YEAR = 2010 #初期の年
FROM_MONTH = 3 #初期の月
EFFECTIVE_RATE = 0.3 # 有効率：取得する日数の割合
FIRST_STOCKS = 1000 # 在庫数の初期値

# データベースの作成

In [3]:
# 日付のリスト生成()
date_list = [datetime(FROM_YEAR, FROM_MONTH, 1) + timedelta(days=i) for i in range(DAYS)]
db = pd.DataFrame({"date": date_list})

# 入庫と出荷を追加
db["warehousing"] = [random.randint(0,20) for i in range(len(db))]
db["shipping"] = [random.randint(0,20) for i in range(len(db))]

# ランダムに日にちを絞り，並び替える
effective_list = random.sample([i for i in range(len(db))], int(EFFECTIVE_RATE * len(db))) 
db = db.iloc[effective_list, :]

db

,date,warehousing,shipping
3439,2019-07-31,14,14
2825,2017-11-24,10,2
3367,2019-05-20,10,0
2132,2016-01-01,5,6
1465,2014-03-05,10,19
...,...,...,...
1184,2013-05-28,20,6
3733,2020-05-20,1,11
1183,2013-05-27,5,2
835,2012-06-13,10,0


# ロジックありの場合

In [4]:
# 年月の範囲を修得する
db_sort = db.sort_values("date")
first_year = db_sort.iloc[0].date.year
first_month = db_sort.iloc[0].date.month
last_year = db_sort.iloc[-1].date.year
last_month = db_sort.iloc[-1].date.month
print(first_year, first_month, last_year, last_month)

# 範囲のリストを生成する．
year_list = list(range(first_year, last_year + 1))
month_list = list(range(1, 13)) * len(year_list)
month_list = month_list[first_month-1: -(12-last_month)]
year_list_all = []
if len(year_list) >= 2:
    year_list_all += [year_list[0]] * (12 - first_month + 1)
    for i in range(len(year_list) - 2):
        year_list_all += [year_list[i+1]] * 12
    year_list_all += [year_list[-1]] * last_month
elif len(year_list) == 1:
    year_list_all += [year_list[0]] * len(month_list)

year_month_list = []
for year, month in zip(year_list_all, month_list):
    year_month_list.append(str(year) + "-" + str(month))

print(year_list, month_list, len(month_list))
print(year_list_all, len(year_list_all))
print(year_month_list)

2010 3 2021 2
[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021] [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2] 132
[2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016

In [5]:
# 識別器
def identification(date):
    # stock_type：0=last_stocks, 1=ten_stocks, 2=twenty_stocks
    if date.day >= 1 and date.day <= 10:
        stock_type = 0
    elif date.day >= 11 and date.day <= 20:
        stock_type = 1
    else:
        stock_type = 2
    # year_month
    year_month = str(date.year) + "-" + str(date.month)
    return stock_type, year_month

last_stocks_dict = dict.fromkeys(year_month_list, 0)
ten_stocks_dict = dict.fromkeys(year_month_list, 0)
twenty_stocks_dict = dict.fromkeys(year_month_list, 0)
stocks_on_the_day_dict = dict.fromkeys(year_month_list, 0)
for a, b in db.iterrows():
    date = b.date.date()
    warehousing = b.warehousing
    shipping = b.shipping
    # 識別器
    stock_type, year_month = identification(date)
    # 加算
    stocks_on_the_day_dict[year_month] += warehousing
    if stock_type == 0:
        last_stocks_dict[year_month] += warehousing - shipping
    elif stock_type == 1:
        ten_stocks_dict[year_month] += warehousing - shipping
    elif stock_type == 2:
        twenty_stocks_dict[year_month] += warehousing - shipping

print("　前月末残：{}\n当月10日残：{}\n当月20日残：{}\n　当月入庫：{}".format(last_stocks_dict, ten_stocks_dict, twenty_stocks_dict, stocks_on_the_day_dict))

　前月末残：{'2010-3': -26, '2010-4': -15, '2010-5': -8, '2010-6': 29, '2010-7': 29, '2010-8': -23, '2010-9': 13, '2010-10': -33, '2010-11': -4, '2010-12': 6, '2011-1': -6, '2011-2': -19, '2011-3': -4, '2011-4': -17, '2011-5': -27, '2011-6': -20, '2011-7': 1, '2011-8': -8, '2011-9': 35, '2011-10': -13, '2011-11': -9, '2011-12': -9, '2012-1': 16, '2012-2': -11, '2012-3': 0, '2012-4': 0, '2012-5': -27, '2012-6': 26, '2012-7': 29, '2012-8': 4, '2012-9': 0, '2012-10': 6, '2012-11': 3, '2012-12': -10, '2013-1': 3, '2013-2': -4, '2013-3': 3, '2013-4': -3, '2013-5': 1, '2013-6': -6, '2013-7': -18, '2013-8': 17, '2013-9': -14, '2013-10': -9, '2013-11': -17, '2013-12': -19, '2014-1': 25, '2014-2': -19, '2014-3': 6, '2014-4': 0, '2014-5': -2, '2014-6': -11, '2014-7': 0, '2014-8': -4, '2014-9': 8, '2014-10': 11, '2014-11': -18, '2014-12': 11, '2015-1': -4, '2015-2': 8, '2015-3': 10, '2015-4': -4, '2015-5': -8, '2015-6': 5, '2015-7': -27, '2015-8': -28, '2015-9': 6, '2015-10': -5, '2015-11': 7, '2015-12

In [6]:
# 増減表の作成
keys = list(last_stocks_dict.keys())
last_stocks_list = list(last_stocks_dict.values())
ten_stocks_list = list(ten_stocks_dict.values())
twenty_stocks_list = list(twenty_stocks_dict.values())
stocks_on_the_day_list = list(stocks_on_the_day_dict.values())

storage_fee_dict = {
    "year-month": keys,
    "last_stocks": last_stocks_list,
    "ten_stocks": ten_stocks_list,
    "twenty_stocks": twenty_stocks_list,
    "stocks_on_the_day": stocks_on_the_day_list
}
storage_fee_db = pd.DataFrame(storage_fee_dict)

storage_fee_db

,year-month,last_stocks,ten_stocks,twenty_stocks,stocks_on_the_day
0,2010-3,-26,-10,19,65
1,2010-4,-15,-3,4,79
2,2010-5,-8,13,-5,121
3,2010-6,29,3,22,92
4,2010-7,29,15,-21,106
...,...,...,...,...,...
127,2020-10,19,6,-13,103
128,2020-11,6,-6,-16,74
129,2020-12,0,3,32,55
130,2021-1,-22,-25,-25,56


In [7]:
stocks = FIRST_STOCKS
for year_month in keys:
    last_stocks_dict[year_month] += stocks
    stocks = last_stocks_dict[year_month]
    ten_stocks_dict[year_month] += stocks
    stocks = ten_stocks_dict[year_month]
    twenty_stocks_dict[year_month] += stocks
    stocks = twenty_stocks_dict[year_month]


last_stocks_list = list(last_stocks_dict.values())
ten_stocks_list = list(ten_stocks_dict.values())
twenty_stocks_list = list(twenty_stocks_dict.values())
stocks_on_the_day_list = list(stocks_on_the_day_dict.values())

storage_fee_dict = {
    "year-month": keys,
    "last_stocks": last_stocks_list,
    "ten_stocks": ten_stocks_list,
    "twenty_stocks": twenty_stocks_list,
    "stocks_on_the_day": stocks_on_the_day_list
}
storage_fee_db = pd.DataFrame(storage_fee_dict)

# 積数
storage_fee_db["sekisu"] = storage_fee_db["last_stocks"] + storage_fee_db["ten_stocks"] + storage_fee_db["twenty_stocks"] + storage_fee_db["stocks_on_the_day"]
print("積数の合計：", storage_fee_db["sekisu"].sum())
storage_fee_db

積数の合計： 384980


,year-month,last_stocks,ten_stocks,twenty_stocks,stocks_on_the_day,sekisu
0,2010-3,974,964,983,65,2986
1,2010-4,968,965,969,79,2981
2,2010-5,961,974,969,121,3025
3,2010-6,998,1001,1023,92,3114
4,2010-7,1052,1067,1046,106,3271
...,...,...,...,...,...,...
127,2020-10,1231,1237,1224,103,3795
128,2020-11,1230,1224,1208,74,3736
129,2020-12,1208,1211,1243,55,3717
130,2021-1,1221,1196,1171,56,3644


In [8]:
print(db.warehousing.sum(), db.shipping.sum(), db.warehousing.sum()-db.shipping.sum())

12031 11842 189
